# **Tarea Nº 4**

- Fecha y hora de entrega: Domingo 12 de enero, 11:59pm
- Agregue los nombres de las personas con las que discutió esta tarea: __Jhoany Rodriguez, Andre Paredes y Fabian Galarza__
- Envíe su tarea haciendo el `push` de su código a su repo en GitHub Classroom: 

## **Mercado de autos usados**

Importa el conjunto de datos `neoauto_20240924.csv`
    

1. ¿Cuál es el precio promedio de los autos de cada marca (`item_brand`) publicados en cada subcategoría (`item_category_2`)?

In [3]:
import pandas as pd
df_neoauto = pd.read_csv('neoauto_20240924.csv')
mean_value = df_neoauto.groupby(['item_brand', 'item_category_2'], as_index=False)['value'].mean()
print(mean_value)

       item_brand item_category_2          value
0      ALFA-ROMEO       Hatchback    8500.000000
1      ALFA-ROMEO           Sedan    2500.000000
2    ASTON MARTIN  Camionetas Suv       0.000000
3    ASTON MARTIN       Deportivo       0.000000
4    ASTON MARTIN           Sedan  149900.000000
..            ...             ...            ...
197         VOLVO  Camionetas Suv   41021.219512
198         VOLVO       Deportivo    8800.000000
199         VOLVO       Hatchback   12272.500000
200         VOLVO           Sedan   10802.777778
201         VOLVO   Station Wagon    6100.000000

[202 rows x 3 columns]


2. ¿Cuántos autos tienen más de 100,000 kilómetros (`item_km`) pero un precio (`item_price`) menor al promedio general?

In [5]:
precio_promedio_general = df_neoauto['item_price'].mean()
autos_filtro = df_neoauto[(df_neoauto['item_km'] > 100000) & (df_neoauto['item_price'] < precio_promedio_general)]
cantidad_autos = autos_filtro.shape[0]
print(cantidad_autos)

497


3. Encuentra el modelo (`item_name`) más caro y el más barato de cada marca (`item_brand`).

In [6]:
modelo_mas_caro_barato = df_neoauto.groupby('item_brand').agg(
    modelo_mas_caro=('item_name', lambda x: x[df_neoauto.loc[x.index, 'item_price'].idxmax()]),
    precio_mas_caro=('item_price', 'max'),
    modelo_mas_barato=('item_name', lambda x: x[df_neoauto.loc[x.index, 'item_price'].idxmin()]),
    precio_mas_barato=('item_price', 'min'))
print(modelo_mas_caro_barato)

                       modelo_mas_caro  precio_mas_caro  \
item_brand                                                
ALFA-ROMEO         alfa-romeo 147 2006           8500.0   
ASTON MARTIN     aston martin dbx 2021         229000.0   
AUDI                    audi rs 6 2024         220000.0   
AUSTIN         austin mini cooper 1970              1.0   
BAIC                     baic x55 2021          12800.0   
...                                ...              ...   
SWM                   swm g05 pro 2022          14000.0   
TOYOTA               toyota hilux 2019         100000.0   
TRIUMPH              triumph tr-7 1971          18900.0   
VOLKSWAGEN    volkswagen teramont 2023          48900.0   
VOLVO                 volvo xc-90 2023          84990.0   

                           modelo_mas_barato  precio_mas_barato  
item_brand                                                       
ALFA-ROMEO           alfa-romeo alfasud 1982             2500.0  
ASTON MARTIN          aston martin

4. Encuentra las tres marcas con mayor número de autos financiados por Santander (`item_financed_by`). 

In [9]:
marcas_top_financiadas = df_neoauto[df_neoauto['item_financed_by'] == 'Santander'].groupby('item_brand')['item_brand'].count().nlargest(3)
print("Las tres marcas con mayor número de autos financiados por Santander son:")
print(marcas_top_financiadas)


Las tres marcas con mayor número de autos financiados por Santander son:
item_brand
BMW       296
TOYOTA    261
NISSAN    228
Name: item_brand, dtype: int64


5. Agrupa las publicaciones por tipo de transmisión (`item_transmission`) y calcula la desviación estándar del precio (`item_price`) para cada tipo.  

In [14]:
desvest_precio_transmision = df_neoauto.groupby('item_transmission')['item_price'].std()
print("Desviación estándar del precio por tipo de transmisión:")
print(desvest_precio_transmision)


Desviación estándar del precio por tipo de transmisión:
item_transmission
Automática                 15332.729048
Automática - Secuencial    22678.641589
Mecánica                    8137.122127
Name: item_price, dtype: float64


6. ¿Cuál es el kilometraje promedio de las publicaciones con el tag "Premium" y de las publicaciones con el tag "Como nuevo"?

In [15]:
kilometraje_tags = df_neoauto[df_neoauto['item_tag'].isin(['Premium', 'Como nuevo'])].groupby('item_tag')['item_km'].mean()
print("Kilometraje promedio por tag:")
print(kilometraje_tags)


Kilometraje promedio por tag:
item_tag
Como nuevo    35963.476190
Premium       48400.235897
Name: item_km, dtype: float64


7. ¿Cuáles son las marcas de vehículos para los que todos sus modelos fabricados en los últimos 5 años tienen al menos 5 publicaciones cada una?

In [37]:
from datetime import datetime

publicaciones_validas = df_neoauto[df_neoauto['item_price'].notna()]

año_actual = 2024 # Asumimos que el año actual es 2024
año_limite = año_actual - 5

autos_recientes = publicaciones_validas[publicaciones_validas['item_year'] >= año_limite]

publicaciones_por_modelo = autos_recientes.groupby(['item_brand', 'item_name']).size().reset_index(name='count') # publicaciones únicas por marca y modelo

modelos_con_mas_de_5_publicaciones = publicaciones_por_modelo[publicaciones_por_modelo['count'] >= 5]

modelos_por_marca = modelos_con_mas_de_5_publicaciones.groupby('item_brand')['item_name'].nunique().reset_index(name='num_modelos') # se agrupa por marca y se cuenta cuántos modelos cumplen el criterio

modelos_totales_recientes = autos_recientes.groupby('item_brand')['item_name'].nunique().reset_index(name='total_modelos') # se cuentan todos los modelos recientes (últimos 5 años) por marca

marcas_validas = pd.merge(modelos_por_marca, modelos_totales_recientes, on='item_brand')

marcas_finales = marcas_validas[marcas_validas['num_modelos'] == marcas_validas['total_modelos']]['item_brand'] # filtro: TODOS los modelos fabricados en los últimos 5 años tienen al menos 5 publicaciones cada una

print("Marcas con todos sus modelos recientes (últimos 5 años) con al menos 5 publicaciones válidas:")
print(marcas_finales.tolist()) # no existen marcas que cumplan con el requisito

Marcas con todos sus modelos recientes (últimos 5 años) con al menos 5 publicaciones válidas:
[]


## **Evolución económica internacional**

Importa el conjunto de datos de la hoja "Data" del archivo `pwt1001.xlsx`. El diccionario de variables está disponible en la hoja "Legend" del archivo Excel.
    

1. Calcule el PBI per cápita real ajustado por poder de paridad de compra (PPP). Use la variable de PBI que se calcula por gasto y la población. ¿Cuáles son los 10 países top?

In [ ]:
# 1. 
# Cargar los datos
file_path = "C:/Users/HP/Downloads/pwt1001.xlsx"
data = pd.read_excel(file_path, sheet_name="Data")
# Calcular el PBI per cápita real PPP
data['rgdpe_pc'] = data['rgdpe'] / data['pop']
# Eliminar valores faltantes
valid_data = data.dropna(subset=['rgdpe_pc', 'year'])
# Obtener el año más reciente
latest_year = valid_data['year'].max()
# Identificar los top 10 países
top_countries = (valid_data[valid_data['year'] == latest_year]
                .nlargest(10, 'rgdpe_pc')
                [['country', 'year', 'rgdpe_pc']]
                .reset_index(drop=True))

print(f"\nTop 10 países por PBI per cápita PPP ajustado (Año {int(latest_year)}):")
print("\nRanking  País                  PBI per cápita PPP")
print("-" * 50)
for idx, row in top_countries.iterrows():
    print(f"{idx+1:<8} {row['country']:<20} {row['rgdpe_pc']:,.2f}")

2. Genera el ránking de los países con el mayor incremento del PBI per cápita real PPP entre los años 1990 y 2019. ¿Dónde se ubica Perú? Para los países que no tengan información para dichos años, puede utilizar datos provenientes de hasta dos años antes o después

In [ ]:
# 2.
data_1990_2019 = valid_data[(valid_data['year'] >= 1988) & (valid_data['year'] <= 2021)]
df_changes = data_1990_2019.pivot(index='country', columns='year', values='rgdpe_pc')
df_changes['change'] = df_changes[2019] - df_changes[1990]
ranking_changes = df_changes.sort_values('change', ascending=False).reset_index()
ranking_peru = ranking_changes[ranking_changes['country'] == 'Peru'].index[0] + 1

print("Ranking de países con mayor incremento (1990-2019):")
print(ranking_changes[['country', 'change']].head(10))
print(f"\nUbicación de Perú: {ranking_peru}° lugar")

3. Para el periodo 2010-2019, calcule el promedio del índice del capital humano. ¿Qué relación existe entre esta variable y el PBI per cápita real PPP calculado previamente? ¿y el número de horas laboradas al año por los trabajadores?

In [ ]:
# 3.
data_2010_2019 = valid_data[(valid_data['year'] >= 2010) & (valid_data['year'] <= 2019)]
human_capital_avg = data_2010_2019.groupby('country')['hc'].mean().reset_index()
relation_df = data_2010_2019.groupby('country').agg({
    'rgdpe_pc': 'mean',
    'hc': 'mean',
    'avh': 'mean'
}).reset_index()
correlacion_hc_gdp = relation_df['hc'].corr(relation_df['rgdpe_pc'])
correlacion_hc_avh = relation_df['hc'].corr(relation_df['avh'])
print("Promedio del índice de capital humano por país (2010-2019):")
print(human_capital_avg.sort_values('hc', ascending=False).head(10))
print("\nCorrelaciones:")
print(f"Capital Humano - PBI per cápita: {correlacion_hc_gdp:.3f}")
print(f"Capital Humano - Horas trabajadas: {correlacion_hc_avh:.3f}")

4. A partir de la variable del precio de los hogares, calcula los países y los años en los cuales se registró la mayor inflación anual para todo el periodo de tiempo disponible en la base de datos

In [ ]:
# 4.
data_sorted = data.sort_values(['country', 'year'])
data_sorted['inflation'] = data_sorted.groupby('country')['pl_con'].pct_change() * 100
data_sorted = data_sorted.replace([float('inf'), float('-inf')], float('nan'))
max_inflation = data_sorted.dropna(subset=['inflation'])
top_inflation = (max_inflation
                .sort_values('inflation', ascending=False)
                .head(10)[['country', 'year', 'inflation']]
                .reset_index(drop=True))
print("Top 10 casos de mayor inflación anual registrada:")
print("\nPaís\t\tAño\tInflación (%)")
print("-" * 40)
for _, row in top_inflation.iterrows():
    print(f"{row['country']:<15} {int(row['year'])}\t{row['inflation']:,.1f}%")

5. ¿Cuáles son los países con la mayor cantidad de missing values en la variable de productividad total de factores a precios corrientes?

In [ ]:
# 5.
missing_tfp = (data
               .groupby('country')['ctfp']  # Agrupamos por país y usamos la variable ctfp
               .apply(lambda x: x.isna().sum())  # Contamos NaN
               .sort_values(ascending=False)  # Ordenamos de mayor a menor
               .reset_index(name='missing_count'))  # Reiniciamos el índice
total_years = data.groupby('country').size().reset_index(name='total_years')
missing_tfp = (missing_tfp
               .merge(total_years, on='country')
               .assign(missing_percentage = lambda x: (x.missing_count / x.total_years * 100)))
print("Países con mayor cantidad de valores faltantes en PTF:")
print("\nPaís\t\t\tValores Faltantes\tPorcentaje\tTotal Años")
print("-" * 70)
for _, row in missing_tfp.head(15).iterrows():
    print(f"{row['country']:<20} {int(row['missing_count']):<20} {row['missing_percentage']:,.1f}%\t\t{row['total_years']}")

6.  Agrupa los datos por país y calcula el promedio de las horas trabajadas anuales (avh) y de la educación promedio (hc) para el periodo 2000-2020.

In [ ]:
# 6.
data_2000_2020 = valid_data[(valid_data['year'] >= 2000) & (valid_data['year'] <= 2020)]
# Calcular promedios por país
country_averages = (data_2000_2020
                   .groupby('country')
                   .agg({
                       'avh': 'mean',  # promedio de horas trabajadas
                       'hc': 'mean',   # promedio de capital humano (educación)
                       'year': ['count', 'min', 'max']  # estadísticas de años disponibles
                   })
                   .round(2)  # redondear a 2 decimales
                   .reset_index())
# Renombrar columnas para claridad
country_averages.columns = ['country', 'avg_hours', 'avg_education', 'years_count', 'start_year', 'end_year']
# Ordenar por horas trabajadas descendente
country_averages_sorted = country_averages.sort_values('avg_hours', ascending=False)

print("\nPromedios por país (2000-2020):")
print("\nPaís                  Horas Trabajadas    Índice Educación    Periodo")
print("-" * 75)
for _, row in country_averages_sorted.iterrows():
    print(f"{row['country']:<20} {row['avg_hours']:>10,.1f}        {row['avg_education']:>8.2f}          {int(row['start_year'])}-{int(row['end_year'])}")

# Calcular algunas estadísticas descriptivas
print("\nEstadísticas descriptivas:")
print("\nHoras trabajadas:")
print(country_averages['avg_hours'].describe().round(2))
print("\nÍndice de educación:")
print(country_averages['avg_education'].describe().round(2))

7.  Genere una variable que ordene a los países según el porcetaje de formación brutal de capital del 2019. Luego, estable diez grupos a manera de deciles. ¿En qué decil se ubica Péru? Vuelva a calcular usando la información de 1990. ¿El Perú se mantiene en el mismo décil o cambia?

In [ ]:
# 7. 
# Función para calcular deciles y posición de Perú
def calcular_deciles(data, año):
    # Filtrar datos del año específico
    data_año = data[data['year'] == año].copy()
    # Calcular el porcentaje de formación bruta de capital
    data_año['fbk_percent'] = data_año['csh_i'] * 100
    # Ordenar países y asignar deciles (1-10)
    data_año['decil'] = pd.qcut(data_año['fbk_percent'], q=10, labels=range(1,11))
    # Obtener posición de Perú
    peru_data = data_año[data_año['country'] == 'Peru']
    # Ordenar países por porcentaje para ver ranking completo
    ranking = data_año.sort_values('fbk_percent', ascending=False)
    return peru_data, ranking
# Calcular para 2019
peru_2019, ranking_2019 = calcular_deciles(valid_data, 2019)
# Calcular para 1990
peru_1990, ranking_1990 = calcular_deciles(valid_data, 1990)

print("Análisis de Formación Bruta de Capital:")
print("\nAño 2019:")
print(f"Perú se ubica en el decil: {peru_2019['decil'].iloc[0]}")
print(f"Porcentaje de FBK: {peru_2019['fbk_percent'].iloc[0]:.2f}%")

print("\nAño 1990:")
print(f"Perú se ubica en el decil: {peru_1990['decil'].iloc[0]}")
print(f"Porcentaje de FBK: {peru_1990['fbk_percent'].iloc[0]:.2f}%")

# Mostrar rankings completos
print("\nRanking 2019 (Top 10):")
print(ranking_2019[['country', 'fbk_percent', 'decil']].head(10))

print("\nRanking 1990 (Top 10):")
print(ranking_1990[['country', 'fbk_percent', 'decil']].head(10))

# Mostrar cambio en la posición relativa
cambio = peru_2019['decil'].iloc[0] - peru_1990['decil'].iloc[0]
print(f"\nCambio en la posición de Perú:")
if cambio == 0:
    print("Perú se mantiene en el mismo decil")
elif cambio > 0:
    print(f"Perú subió {abs(cambio)} deciles")
else:
    print(f"Perú bajó {abs(cambio)} deciles")

## **Transacciones Financieras**

Importa el conjunto de datos `base_financiera.csv`
    

8. Filtra todas las transacciones de tipo "Debit" en las que el monto sea mayor a 200, y calcula el monto total de esas transacciones. Además, determina cuántos clientes únicos realizaron esas transacciones.

In [14]:

import pandas as pd

#Importamos la base de datos "Base financiera" en formato CSV desde el terminal local:
data_base = "C:\GitHub - Q -LAB\tarea-4-rodriguez-paredes-galarza\base_financiera.csv"

# Guardamos la Base de datos en una variable con Pandas.
data = pd.read_csv('base_financiera.csv')

# Filtramos las transacciones de tipo "Debit" con monto amyores a 200
filtered_data = data[(data['Transaction_Type'] == 'Debit') & (data['Amount'] > 200)]

# Calculamos el número total de variaciones en los datos filtrados
total_amount = filtered_data['Amount'].sum()

# Determinamos el número de clientes únicos
unique_customers = filtered_data['Customer_ID'].nunique()

# Resultados
print(f"Monto total de transacciones: {total_amount}")
print(f"Número de clientes únicos: {unique_customers}")


Monto total de transacciones: 2847871.75
Número de clientes únicos: 200


9. Identifica las transacciones con valores nulos en la columna (`Amount`) . Rellena esos valores con la mediana de los montos por tipo de transacción (`Transaction_Type`) y luego calcula el monto total de transacciones por región (`region`) .

In [17]:
import pandas as pd

#Importamos la base de datos "Base financiera" en formato CSV desde el terminal local:
data_base = r"C:\GitHub - Q -LAB\tarea-4-rodriguez-paredes-galarza\base_financiera.csv"
# Cargamos la base de datos desde el archivo CSV
data = pd.read_csv(data_base)

# Identificamos transacciones con valores nulos en la columna "Amount":
missing_values = data[data['Amount'].isnull()]
print("Transacciones con valores nulos en 'Amount':")
print(missing_values)

# Calculamos la mediana del monto por tipo de transacción:
medians_by_type = data.groupby('Transaction_Type')['Amount'].median()

# Rellenamos valores nulos con la mediana:
data['Amount'] = data.apply(
    lambda row: medians_by_type[row['Transaction_Type']] if pd.isnull(row['Amount']) else row['Amount'], axis=1
)

# Verificamos que no hay valores nulos:
print("\nValores nulos después de rellenar:")
print(data['Amount'].isnull().sum())

# Calculamos el monto total de transacciones por región:
total_by_region = data.groupby('Region')['Amount'].sum()

# Mostrar resultados:
print("\nMonto total de transacciones por región:")
print(total_by_region)


Transacciones con valores nulos en 'Amount':
      Transaction_ID Customer_ID        Date  Amount Transaction_Type Region
34              1035        C043  2023-02-04     NaN            Debit   East
40              1041        C051  2023-01-01     NaN           Credit   West
60              1061        C086  2023-01-21     NaN            Debit   East
66              1067        C179  2023-01-27     NaN           Credit   East
82              1083        C076  2023-01-03     NaN            Debit  South
...              ...         ...         ...     ...              ...    ...
1944            2945        C134  2023-01-25     NaN           Credit   West
1967            2968        C150  2023-01-08     NaN            Debit  South
2015            1276        C047  2023-02-05     NaN           Credit   West
2018            2647        C068  2023-01-07     NaN            Debit  North
2023            2084        C123  2023-01-04     NaN            Debit   East

[113 rows x 6 columns]

Valore

10. Busca transacciones duplicadas considerando las columnas (`Customer_ID`) y (`Date`). Elimina los duplicados manteniendo solo la primera ocurrencia. Luego, agrupa las transacciones restantes por (`Customer_ID`) y calcula el monto promedio y la cantidad de transacciones por cliente.

In [18]:
import pandas as pd

data_base = r"C:\GitHub - Q -LAB\tarea-4-rodriguez-paredes-galarza\base_financiera.csv"

data = pd.read_csv(data_base)

# Buscamos transacciones duplicadas considerando las columnas "Customer_ID" y "Date"
duplicates = data.duplicated(subset=['Customer_ID', 'Date'], keep='first')
print(f"Número de transacciones duplicadas: {duplicates.sum()}")

# Eliminamos duplicados, manteniendo solo la primera ocurrencia
data_cleaned = data.drop_duplicates(subset=['Customer_ID', 'Date'], keep='first')

# Agrupamos por "Customer_ID" y calculamos monto promedio y cantidad de transacciones por cliente
grouped_data = data_cleaned.groupby('Customer_ID').agg(
    Average_Amount=('Amount', 'mean'),
    Transaction_Count=('Transaction_ID', 'count')
).reset_index()

# Mostramos resultados
print("\nDatos agrupados por cliente:")
print(grouped_data)

# COMO ADICONAL: Guardamos la base de datos "limpiada" (con los cambios):
grouped_data.to_csv(r"C:\GitHub - Q -LAB\tarea-4-rodriguez-paredes-galarza\resultados_por_cliente.csv", index=False)


Número de transacciones duplicadas: 234

Datos agrupados por cliente:
    Customer_ID  Average_Amount  Transaction_Count
0          C001     3334.711875                 17
1          C002     1881.827778                  9
2          C003     3102.125000                 14
3          C004     3053.882000                  7
4          C005     3364.053333                  6
..          ...             ...                ...
195        C196     2566.555000                 10
196        C197     2530.674286                 14
197        C198     2484.925000                 12
198        C199     3062.287500                  5
199        C200     2130.977500                  5

[200 rows x 3 columns]
